# Setup 

In [2]:
import os
import pandas as pd
from tecton_gen_ai.fco import source_as_knowledge, prompt, AgentService, AgentClient
from tecton_gen_ai.testing import make_local_source, set_dev_mode
from tecton_gen_ai.testing.utils import make_local_vector_db_config

set_dev_mode()



# Setup Sample Feature Views

In [3]:
from tecton_gen_ai.testing import make_local_batch_feature_view

# user specific money transfer statistics
transfer_stats = make_local_batch_feature_view(
    "transfer_stats",
    [
        {"user_id": 1, "transfers_in_last_7_days": 20, "transfers_in_last_1_year": 22},
        {"user_id": 2, "transfers_in_last_7_days": 0, "transfers_in_last_1_year": 10},
    ],
    ["user_id"],
    description = "User's money transfer stats, abnormal activities could cause account suspension"  # description for LLM
)

# user profile info
user_profile = make_local_batch_feature_view(
    "user_profile",
    [
        {"user_id": 1, "name": "Jim", "age": 30, "account_status":"suspended"},
        {"user_id": 2, "name": "Mary", "age": 16, "account_status":"active"},
    ],
    ["user_id"],
    description = "User's name, age and account status"  # description for LLM
)


# Chatbot for Money Transfer App

This money transfer service app uses a chatbot that responds to user questions based on their public FAQ.

The chatbot uses a RAG solution.

<Insert image with high level RAG architecture.>

## The FAQ Data

As expected the FAQ data is text with common user questions and answers.

In [4]:
faq_df = pd.read_parquet("faq.parquet")
display (faq_df)

,question,answer
0,Am I eligible to receive a transfer?,You’re eligible to receive a transfer from Rem...
1,Can I get my Remitly account back?,If you think your profile may have been suspen...
2,Can I get my account back?,"In most cases, it’s easy to get back into your..."
3,Can I receive money using mobile wallets or ho...,You can use home delivery or mobile wallet app...
4,Can I send a request via fax?,We are not able to accept subpoenas or other l...
...,...,...
241,Why is my transfer being reviewed by Bancolombia?,There are a couple of reasons Bancolombia migh...
242,Why was my sending limit increase declined?,"In some cases, we may not be able to approve t..."
243,Will I need to change my sending country befor...,"No, your sending country is the country where ..."
244,Will my info be deleted?,Closing your profile doesn’t delete your infor...


## Adding FAQ knowledge for RAG

The FAQ text is loaded into a vector search database to provide answers to user's questions.

In [5]:
# The source description tells the LLM what kind of information it can retrieve from this knowledge base
src = make_local_source(
    "faq",
    faq_df,
    description = "FAQ for TectonTransfer users",  # <<<<<<<<
    max_rows = len(faq_df)
)

# calculate embeddings based on the "question" such that LLM can find info it needs based on similar questions
faq_knowledge = source_as_knowledge(
    src,
    vector_db_config=make_local_vector_db_config(),
    vectorize_column="question" # <<<<<<<<
)

# the prompt instructs the LLM to use the FAQ knowledge to answer questions 
@prompt()
def sys_prompt() -> str:
    return """You are an assistant,
    TectonTransfer is a money transfer service on a mobile application.
you answer questions based on FAQ for TectonTranfer.
Say you don't know if the questions are not relevant to any questions on FAQ
"""

## The AgentService 

AgentService provides context to the LLM by providing the system prompt and sets of knowledge.

In [7]:
rag_service = AgentService(
    "chatbot_context",
    prompts=[sys_prompt],
    knowledge=[faq_knowledge]
)

rag_client = AgentClient.from_local(rag_service)

UserCodeError: error caused by user configuration has occurred:
Pandas pipeline function (feature view 'faq_batch') failed with exception
1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

## Add an LLM

In [22]:
from tecton_gen_ai.testing.interactive import auto_complete, qna
from langchain_openai import ChatOpenAI

openai = ChatOpenAI(model = "gpt-4o", temperature=0)


## A RAG-based chatbot

No personalization, all users get the same experience.

In [23]:
qna(rag_client, openai, "sys_prompt")

# Personalized Chatbot

A more useful chatbot can be created by adding better context and tools for the LLM.

insert image with data pipelines for 
 - transaction stats aggregation  with `transfers_last_7days` & `transfers_last_1year`
 - user profile with `name`, `age` & `account_status`
 - With Tecton -> the data is always up to date, 
 - Agent delivers 
    - FAQ based knowledge
    - contextualized prompt, 
    - user and transaction data tools to answer based on user's situation 

In [24]:
from tecton_gen_ai.fco import tool


# this prompt incorporates user_profile context in the prompt  
@prompt(sources=[user_profile])
def sys_prompt_fv(user_profile) -> str:
    return f"""
You are an assistant, 
You answer questions based on FAQ list.
Say you don't know if the questions are not relevant to any questions on FAQ.
You are serving {user_profile['name']}, whose age is {user_profile['age']}.
Address the user by name.
Consult the user's money transfer stats if needed to respond to their question.
"""
    
# a service with user context for personalization
service_with_fv = AgentService(
    "chatbot_with_personalization",
    prompts=[sys_prompt_fv],
    knowledge=[faq_knowledge],
    tools=[transfer_stats, user_profile],  # <<<<<< the LLM can now query this user info as needed
)

client_with_fv = AgentClient.from_local(service_with_fv)

## A chatbot with user context

The chatbot identifies the user with a `user_id` based on their session.

Tecton makes use of the `user_id` to personalize the interaction.

In [26]:
# the chatbot has the user identified based on their session
qna(client_with_fv, openai, "sys_prompt_fv", context={"user_id":2}) 